# Randomized Linear Algebra
![title](machine_learning.png)

Linear algebra forms the bedrock of many machine learning algorithms, providing the essential tools for data manipulation and analysis.  For example, linear regression involves the solutin of a linear system of equations to determine optimal coefficients defining the relationship between variables.  In clustering algorithms, especially those relying on distance metrics like k-means, linear algebra facilitates the calculation of distances between data points, enabling the grouping of similar items.  Principal component analysis (PCA), a dimensionality reduction technique, heavily relies on eigenvalue and eigenvector decompositions of covariance matrices, all core concepts of linear algebra.  Critically, linear algebra provides the framework for representing and manipulating data as vectors and matrices.  

However, oftentimes we work with datasets that are sufficiently large such that the computational complexity of typical linear algebraic operations such as matrix multiplication, inversion, or decomposition are too expensive.  In such circumstances, randomization provides us tools such that we can trade a (sometimes much) faster runtime for a (surprisingly little) reduced accuracy.  In the following notebook, we will explore such randomized methods as applied to the cases of linear regression, k-means, and principal component analysis.  

## Linear regression with radial basis functions
Imagine that we are interested in creating a topographic map based on a collection of measured locations.  Let's define the spatial coordinates $\hat{\mathbf{x}}$ at which the elevation $\hat{y}$ is measured.  Such a collection of measurements might appear as follows:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)
n_data = 2**11
x = np.random.rand(n_data,2)

D2 = (x[:,0] - x[:,0].reshape(-1,1))**2 + (x[:,1] - x[:,1].reshape(-1,1))**2

sigma = 0.3
l = 0.1
K = np.exp(-D2/l**2)
C = np.linalg.cholesky(K + np.eye(n_data)*1e-9)

y = C @ np.random.randn(n_data) + sigma*np.random.randn(n_data)

plt.scatter(*x.T,c=y,vmin=-2,vmax=2)
plt.axis('equal')
plt.colorbar()

Now that we have a dataset, we would like to create an interpolant - a function $y(x)$ that can accept arbitrary $x$ and predict an associated elevation, but that produces sensible behavior with respect to the observations - mainly that close to the observations they agree, and away from observations the solutions exhibit smooth behavior.  One way to create such a function is through radial basis function regression.  Such a method presupposes that the desired function is a sum of radial basis functions
$$
y(\mathbf{x}) = \sum_{k=1}^m \phi_k(\mathbf{x}) w_k,
$$
where 
$$
\phi_k(\mathbf{x}) = \exp\left(-\frac{(\mathbf{x} - \mu_k)^2}{l_k^2}\right), 
$$
is a radial basis function (here a so-called squared exponential), each with a different mean and a length scale that we select a priori.  As an example, a radial basis function with mean $\mu_k=[0.5,0.75]$ and with length scale $l=0.1$, might look like this when evaluated over the test set:

In [ ]:
mu = np.array([0.5,0.75])
l = 0.1
phi_k = np.exp(-((x - mu)**2).sum(axis=1)/l**2)
plt.scatter(*x.T,c=phi_k)

Our solution is just a linear combination of such basis functions.  We suppose that the means are equally spaced in both spatial dimensions, and that they overlap a bit.  We collect the basis functions into a matrix $\Phi(\mathbf{x}) \in \mathbb{R}^{n \times m}$ where each column corresponds to a different basis function, and where the rows correspond to a pointwise evaluation of that function.  We can then write our prediction $y = \Phi(\mathbf{x}) \mathbf{w}$.

In [ ]:
# Create a grid of means
x_ = np.linspace(0,1,11)
y_ = x_
X,Y = np.meshgrid(x_,y_)

means = np.c_[X.ravel(),Y.ravel()]
l = 0.1

# Build the $\Phi$ matrix
Phi = np.column_stack([np.exp(-((x - m)**2).sum(axis=1)/l**2) for m in means])
m = Phi.shape[1]

Just to get a sense for what functions are possible under this model, we could try it out with some random coefficients and evaluated over the training data points we saw before:

In [ ]:
fig,axs = plt.subplots(nrows=2,ncols=2)
axs = axs.ravel()
for ax in axs:
    y_random = Phi @ np.random.randn(Phi.shape[1])
    ax.scatter(*x.T,c=y_random)
    ax.axis('equal')

Note that this is just one way of constructing an interpolant, and it's subject to many arbitrary choices!  Nonetheless, based on the random functions that we generated above, we hope that we ought to be able to reasonably approximate our dataset.   

Of course, *a priori* we don't know what the weight matrix $\mathbf{w}$ is - we will need to constrain it with data.  In particular, we'd like to minimize the squared $L_2$ norm between the model's predictions and the observations, measured by the cost function
$$
\mathcal{L}(\mathbf{w}) = (\Phi(\mathbf{x}) \mathbf{w} - \hat{y})^T\Sigma^{-1}(\Phi(\mathbf{x}) \mathbf{w} - \hat{y})
$$
where $\hat{y}$ are the function values observed at $\mathbf{x}$, and the matrix $\Sigma^{1}$ is - in this case - a diagonal matrix containing the variance in the observations.  In addition, we'd like to constrain the problem so that the values $\mathbf{w}$ don't stray too far from zero, so we add the regularization
$$
\mathcal{P}(\mathbf{w}) = \mathbf{w}^T \Sigma_0^{-1} \mathbf{w},
$$
where $\Sigma_0$ is a (usually diagonal) regularization matrix.  The minimization problem is then
$$
\underset{\mathbf{w}}{\operatorname{argmin}} \; \mathcal{L}(\mathbf{w}) + \mathcal{P}(\mathbf{w})
$$
We immediately recognize this as a typical least squares problem, which has as a solution
$$
\left(\Phi^T \Sigma^{-1} \Phi + \Sigma_0^{-1}\right) \mathbf{w} = \Phi^T \Sigma^{-1} \hat{y}.
$$
We can solve this immediately to get our estimate of $\mathbf{w}$

In [ ]:
class LinearRegression:
    def __init__(self,sigma2=1.0,gamma=1.0):
        self.sigma2 = sigma2

    def fit(self,X,y):
        self.X = X
        self.y = y
        self.w = np.linalg.solve(X.T/self.sigma2 @ X + np.eye(X.shape[1]),X.T/self.sigma2 @ y)

    def predict(self,X):
        return X @ self.w
        
model = LinearRegression(sigma2=sigma**2)
model.fit(Phi,y)
y_exact = model.predict(Phi)
w_exact = model.w

fig,axs = plt.subplots(ncols=2)
fig.set_size_inches(8,4)
axs[0].scatter(*x.T,c=y_exact,vmin=-3,vmax=3)
axs[0].axis('equal')
axs[0].set_title('Model')
axs[1].scatter(*x.T,c=y,vmin=-3,vmax=3)
axs[1].axis('equal')
axs[1].set_title('Observation')

This approximates the data well (with the exception of the noise - our model is much smoother, which is probably a good thing).  How expensive was this?  The matrix $\Phi^T \Sigma^{-1} \Phi$ is $m \times m$, so solving this linear system via direct means is $m^3$.  However, the constructing that matrix costs $n m^2$ to perform the matrix multiplication.  The addition of these two terms yields the expense.

The $n$ term can be prohibitive for some very large problems.  However, if we could reduce the dimensionality of the matrix, we could get a faster algorithm.  For instance, consider solving instead the minimization problem with
$$
\mathcal{L}(\mathbf{w}) = (S \Phi(\mathbf{x}) \mathbf{w} - S \hat{y})^T\Sigma^{-1}(S \Phi(\mathbf{x}) \mathbf{w} - S \hat{y})
$$
for some $S\in\mathbb{R}^{d \times n}$ (Note that to ensure that the relative weighting of the data misfit versus the prior remains the same, we have to scale $S$ appropriately).  If we had the matrix $S \Phi$, then the resulting complexity would only be $\mathcal{O}(dm^2 + m^3)$ (assuming $d$ is $\mathcal{O}(m)$).  Does this work?  Let's find out.  

### Subsampling
First, what should we use as $S$?  We could try subsampling, such that 
$$
S = \sqrt{\frac{n}{d}} I[s],
$$
where $s$ is a set of random indices over $n$.


In [ ]:
d = 200
np.random.seed(0)
indices = np.random.choice(n_data,d,replace=False)
S = np.sqrt(n_data/d)*np.eye(n_data)[indices]

model.fit(S @ Phi,S @ y)
y_subsample = model.predict(Phi)

fig,axs = plt.subplots(ncols=3)
fig.set_size_inches(12,4)
axs[0].scatter(*x.T,c=y_subsample,vmin=-3,vmax=3)
axs[0].axis('equal')
axs[0].set_title('Approx')
axs[1].scatter(*x.T,c=y_exact,vmin=-3,vmax=3)
axs[1].axis('equal')
axs[1].set_title('Exact')
axs[2].scatter(*x.T,c=y_subsample - y_exact,vmin=-1,vmax=1,cmap=plt.cm.seismic)
axs[2].axis('equal')
axs[2].set_title('Diff')


Note that as implemented, this isn't any more efficient, because the matrix product $S \Phi$ takes $\mathcal{O}(d n m)$ time to compute (which is similar to the $\mathcal{O}(nm^2)$ of the exact case).  However, we note that we can take the matrix product $S \Phi$ in time faster than $d n m$ by just extracting the appropriate rows from $\Phi$.   This feels obvious of course, but it speaks to a more general pattern for gaining efficiency, namely choosing $S$ so that evaluating the product $S \Phi$ is cheaper.  

In [ ]:
class SubsampleSketch:
    def __init__(self,d):
        self.d = d

    def sketch(self,A,b=None,seed=None):
        if seed is not None:
            np.random.seed(seed)
        n,m = A.shape
        indices = np.random.choice(n,d,replace=False)  # Choose d random indices from n
        scale = np.sqrt(n/d)
        SA = scale*A[indices]
        if b is not None:
            Sb = scale*b[indices]
            return SA,Sb
        else:
            return SA

d = 200
sketch_subsample = SubsampleSketch(d)

model.fit(*sketch_subsample.sketch(Phi,b=y,seed=0))
y_subsample = model.predict(Phi)

fig,axs = plt.subplots(ncols=3)
fig.set_size_inches(12,4)
axs[0].scatter(*x.T,c=y_subsample,vmin=-3,vmax=3)
axs[0].axis('equal')
axs[0].set_title('Approx')
axs[1].scatter(*x.T,c=y_exact,vmin=-3,vmax=3)
axs[1].axis('equal')
axs[1].set_title('Exact')
axs[2].scatter(*x.T,c=y_subsample - y_exact,vmin=-1,vmax=1,cmap=plt.cm.seismic)
axs[2].axis('equal')
axs[2].set_title('Diff')

Unsurprisingly, the quality of the approximation depends heavily on the number of sampled columns - if we choose $d=n$, then we haven't done any compression at all!  **Experiment a bit with different values of $d$ in the above code to get a sense for how the approximation improves for more data.**



### Rademacher distributed sample matrix
For small $d$, the subsampling described above is obviously extremely lossy: we are just throwing away data!  Perhaps there is a different way to try to do something similar that doesn't waste as much information.  Note that in principle, there shouldn't be any reason why we couldn't use a different choice for the matrix $S$; after all, if we conceptualize the problem as solving a (overdetermined) linear system of equations, then we are multiplying both sides by the same thing.  For example, let's use a matrix where every entry is given by an independent Rademacher distributed random variable (basically a Bernoulli over -1 and 1, rather than 0 and 1, scaled by the root of $n$.  Just as multiplying both sides by a (non-zero) scalar ought to lead to the same solution, it seems intuitive that multiplying both sides by a matrix should lead to close to the same solution.  This also seems like a good idea because it will include all the data in the sketch (even if in an averaged way).  

In [ ]:
class RademacherSketch:
    def __init__(self,d):
        self.d = d

    def sketch(self,A,b=None,seed=None):
        if seed is not None:
            np.random.seed(seed)
        n,m = A.shape
        scale = 1./np.sqrt(self.d)
        S = scale*(np.random.randint(0,2,(self.d,n))*2-1)
        SA = S @ A
        if b is not None:
            Sb = S @ b
            return SA,Sb
        else:
            return SA

sketch_rad = RademacherSketch(d)

model.fit(*sketch_rad.sketch(Phi,b=y,seed=0))
y_rad = model.predict(Phi)

fig,axs = plt.subplots(ncols=3)
fig.set_size_inches(12,4)
axs[0].scatter(*x.T,c=y_rad,vmin=-3,vmax=3)
axs[0].axis('equal')
axs[0].set_title('Approx')
axs[1].scatter(*x.T,c=y_exact,vmin=-3,vmax=3)
axs[1].axis('equal')
axs[1].set_title('Exact')
axs[2].scatter(*x.T,c=y_rad - y_exact,vmin=-1,vmax=1,cmap=plt.cm.seismic)
axs[2].axis('equal')
axs[2].set_title('Diff')

Here we've used the same value for $d$ and the results look okay, but let's get the average error over a number of different trials to compare whether this method actually is better

In [ ]:
n_trials = 200

subsample_errors = []
for j in range(n_trials):

    model.fit(*sketch_subsample.sketch(Phi,b=y))
    error = np.sqrt(((model.predict(Phi) - y_exact)**2).mean())
    subsample_errors.append(error)

rad_errors = []
for j in range(n_trials):

    model.fit(*sketch_rad.sketch(Phi,b=y))
    error = np.sqrt(((model.predict(Phi) - y_exact)**2).mean())
    rad_errors.append(error)

print(np.mean(subsample_errors),np.std(subsample_errors))
print(np.mean(rad_errors),np.std(rad_errors))

We see that the errors in doing this are smaller, and there is greater run-to-run consistency.  Unfortunately, this method doesn't actually lead to much of a benefit.  The reason for this is that the matrix product $S \Phi$ is expensive to compute.  However, maybe we can take a middle ground approach that gets better accuracy than just randomly subsampling, while also allowing a fast matrix product.

## Better random matrix transforms
There are two principal ways that people have sought to efficiently produce sketches.  The first is conceptually easy: take the Rademacher approach as described above, but limit each column to contain only $\zeta$ non-zero entries.  As such the matrix will be sparse, and we can use matrix multiplication routines that capitalize on that sparsity to reduce the expense to $\mathcal{O}(\zeta m n)$.  This sketching technique - which is called the sparse sign embedding - looks something like this:


In [ ]:
from scipy.sparse import lil_array

class SparseSignSketch:
    def __init__(self,d,zeta):
        self.d = d
        self.zeta = zeta

    def sketch(self,A,b=None,seed=None):
        if seed is not None:
            np.random.seed(seed)
        n,m = A.shape
        S = lil_array((self.d,n))
        scale = 1./np.sqrt(self.zeta)
        for i in range(n):
            rows = np.random.choice(self.d,self.zeta,replace=False)
            S[rows,i] = scale*(np.random.randint(0,2,self.zeta)*2 - 1)

        SA = S @ A
        if b is not None:
            Sb = S @ b
            return SA,Sb
        else:
            return SA

zeta = 8
sketch_sparse = SparseSignSketch(d,zeta)

model.fit(*sketch_sparse.sketch(Phi,b=y,seed=0))
y_sparse = model.predict(Phi)

fig,axs = plt.subplots(ncols=3)
fig.set_size_inches(12,4)
axs[0].scatter(*x.T,c=y_sparse,vmin=-3,vmax=3)
axs[0].axis('equal')
axs[0].set_title('Approx')
axs[1].scatter(*x.T,c=y_exact,vmin=-3,vmax=3)
axs[1].axis('equal')
axs[1].set_title('Exact')
axs[2].scatter(*x.T,c=y_sparse - y_exact,vmin=-1,vmax=1,cmap=plt.cm.seismic)
axs[2].axis('equal')
axs[2].set_title('Diff')

In [ ]:
sparse_errors = []
for j in range(n_trials):

    model.fit(*sketch_sparse.sketch(Phi,b=y))
    error = np.sqrt(((model.predict(Phi) - y_exact)**2).mean())
    sparse_errors.append(error)

print(np.mean(subsample_errors),np.std(subsample_errors))
print(np.mean(rad_errors),np.std(rad_errors))
print(np.mean(sparse_errors),np.std(sparse_errors))

We end up with results that don't differ much from the full Rademacher embedding at a substantially lower (asymptotic) cost.  This is, in general, a pretty good method to use by default.  

The other common approach is somewhat more esoteric.  The idea is - in essence - to change the basis of the matrix and perform subsampling in the other basis.  Why is this a good idea?  Consider subsampling as above.  The principle weakeness of this method comes from the fact that we might accidentally sample unimportant points (like two that are right next to one another) or we might miss points that exert a strong control on the data.  The formal metric for deciding the "importance" of a point is called leverage, and sensible greedy algorithms exist for subsampling by leverage score.  Unfortunately, computing leverage is expensive.  The alternative approach is to instead transform the data matrix into a basis that is less local and more global, such as the Fourier or Walsh-Hadamard space, and to perform subsampling there.  Since these basis functions are global (rather than pointwise), their leverage scores tend to be uniformly distributed.  The expense then comes from the change of basis.  However, fast algorithms for changing from the standard to Fourier basis (or other alternatives) exist - these are called Fast Fourier Transforms, and they can perform the basis change in $n \log n$ time.  As such, we have the matrix 
$$
S = R F D,
$$
where $R$ is a subsampling matrix, $F$ is a structured matrix allows for fast application, and $D$ is a diagonal matrix that effects a random sign flip.  Here we'll use the Fast Walsh Hadamard Transform.

In [ ]:
def fwht(x):
    """
    Fast Walsh-Hadamard Transform (FWHT) - recursive implementation.

    Args:
        x (numpy.ndarray): Input array (length must be a power of 2).

    Returns:
        numpy.ndarray: Transformed array.
    """
    N = len(x)
    if N <= 1:
        return x
    else:
        even = fwht(x[0::2])
        odd = fwht(x[1::2])
        return np.concatenate([even + odd, even - odd])


In [ ]:
class HadamardSketch:
    def __init__(self,d):
        self.d = d

    def sketch(self,A,b=None,seed=None):
        if seed is not None:
            np.random.seed(seed)
        n,m = A.shape
        indices = np.random.choice(n,self.d,replace=False)
        D = np.random.randint(0,2,n)*2 - 1
        scale = 1./np.sqrt(d)
        SA = scale*fwht(D.reshape(-1,1)*A)[indices]
        if b is not None:
            Sb = scale*fwht(D.reshape(-1,1)*b.reshape(-1,1))[indices].ravel()
            return SA,Sb
        else:
            return SA

d = 200

sketch_hadamard = HadamardSketch(d)

model.fit(*sketch_hadamard.sketch(Phi,b=y,seed=0))
y_hadamard = model.predict(Phi)

fig,axs = plt.subplots(ncols=3)
fig.set_size_inches(12,4)
axs[0].scatter(*x.T,c=y_hadamard,vmin=-3,vmax=3)
axs[0].axis('equal')
axs[0].set_title('Approx')
axs[1].scatter(*x.T,c=y_exact,vmin=-3,vmax=3)
axs[1].axis('equal')
axs[1].set_title('Exact')
axs[2].scatter(*x.T,c=y_hadamard - y_exact,vmin=-1,vmax=1,cmap=plt.cm.seismic)
axs[2].axis('equal')
axs[2].set_title('Diff')

In [ ]:
print(np.linalg.norm(sketch_hadamard.sketch(Phi),axis=0))
np.linalg.norm(Phi,axis=0)

In [ ]:
hadamard_errors = []
ws = []
for j in range(n_trials):

    model.fit(*sketch_hadamard.sketch(Phi,b=y))
    error = np.sqrt(((model.predict(Phi) - y_exact)**2).mean())
    hadamard_errors.append(error)
    ws.append(model.w)

print(np.mean(subsample_errors),np.std(subsample_errors))
print(np.mean(rad_errors),np.std(rad_errors))
print(np.mean(sparse_errors),np.std(sparse_errors))
print(np.mean(hadamard_errors),np.std(hadamard_errors))

### Why does this work?
While it seems intuitive that this *might* work (at least to me), it's not really clear why it *does* work.  To see why this is the case, we need to get an understanding for what the matrix product changes, and what it leaves the same.  The essential property of all of the matrices that we've considered is the following property called the *subspace embedding property*
$$
\| S A x \| \le (1 + \epsilon) \| A x \|\; \forall x\in\mathbb{R}^m.
$$
Stated in words, the length of any vector $Ax$ remains the same (up to a distortion factor $\epsilon$) when you multiply it by $S$, even though the dimensionality of that vector is usually not the same.  In fact, theory can provide us a (relatively conservative) bound on the number of sketching elements required to achieve a given distortion factor (approximately $d = \frac{\log n}{\epsilon^2}$).  Let's see what we mean by this:

In [ ]:
eps = 0.1
m = Phi.shape[1]
d = int(np.log(n_data)//eps**2)

sketch = HadamardSketch(d)

SA,Sb = sketch.sketch(Phi,y)

v = np.random.randn(m,500)

G1 = SA @ v
G2 = Phi @ v

plt.figure()
plt.plot(np.linalg.norm(G2,axis=0),np.linalg.norm(G1,axis=0),'k.')
plt.plot(np.linalg.norm(G2,axis=0),np.linalg.norm(G2,axis=0)*(1+eps))
plt.plot(np.linalg.norm(G2,axis=0),np.linalg.norm(G2,axis=0)*(1-eps))   

The norms are preserved within the predicted bounds!  How does this interact with linear regression?  Consider the form of the normal equations
$$
(\Phi^T \Phi + \gamma \sigma^2 I) \mathbf{w} = \Phi^T y.
$$
(Note that I've factored out the (diagonal and uniform) observational uncertainty to make the algebra more clear).  Let's look at the product $\Phi^T \Phi$.  Its $k$-th diagonal entry is 
$$
\phi_k^T \phi_k = \|\phi_k\|_2^2.
$$
If norms are preserved, then we can immediately use the fact that the relative difference in norms between the sketched matrix and the original to show that the diagonals in the normal equation are approximately the same.  

In [ ]:
plt.plot(np.diag(Phi.T @ Phi),np.diag(SA.T @ SA),'k.')

Perhaps surprisingly, subspace embeddings also preserve angles.  This (in conjunction with the preservation of norms) implies that *dot products* are generally preserved (of which the norm is the special case of the dot product of a vector with itself).  As such, we would expect all of the entries in $\Phi^T \Phi$ (which are just dot products of rows with columns) to be reasonably close to $(S \Phi)^T (S \Phi)$ (and the same for $\Phi^T y$ versus $(S \Phi)^T Sy$).

In [ ]:
fig,axs = plt.subplots(ncols=3)
fig.set_size_inches(12,4)
axs[0].imshow(Phi.T @ Phi,vmin=0,vmax=10)
axs[1].imshow(SA.T @ SA,vmin=0,vmax=10)
axs[2].plot(Phi.T @ y,SA.T @ Sb,'k.')
axs[2].plot(np.array([-100,100]),np.array([-100,100]))
axs[2].axis('equal')

We can manipulate these properties to also gain a similar bound on errors in predictions - when we make a prediction using coefficients $\mathbf{w}$ derived by solving the sketched least squares problem, the resulting cost is bounded by $1+\epsilon$ times the minimum of the full problem.  However, we do *not* have a strong guarantee on whether the actual values of $\mathbf{w}$ determined from the sketched problem converge to the solution of the unsketched problem.  Indeed, if we solve the sketched problem many times:



In [ ]:
plt.plot(w_exact,np.column_stack(ws),'.')
plt.axis('equal')
plt.xlabel('Exact w')
plt.ylabel('Approximate w')

However, there are alternative methods that allow for this error to be substantially reduced, if that is a requirement for the application.

## Column-space embeddings
In the previous example, we had a large number of data points that we wanted to reduce to a smaller number of "aggregate" points for computational efficiency when solving a linear regression problem.  In solving the normal equations, we saw that geometry was preserved over the reduced rows, the normal equations used to solve the least squares problem was only mildly perturbed.  However, for some problems, we might not be so interested in fitting a curve as we are in categorizing data instances - for example in clustering or logistic regression.  In these cases, it doesn't necessarily make sense to try to aggregate data points - but it might make sense to try to reduce the dimensionality of the feature space, i.e. to covert a data matrix $X\in \mathbb{R}^{n\times m}$ into $\hat{X}\in \mathbb{R}^{n\times d}$.  Perhaps unsurprisingly, we  can do a largely similar thing, but applied to the data matrix's column space instead of its row space, i.e. performing right multiplication instead.  

Let's do clustering first. Let's try applying this to the MNIST digits dataset.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X = mnist.data/255.
plt.imshow(X[0].reshape((28,28)))

We can generate a much compressed version of the mnist dataset for $k-$means clustering by simply right multiplying by a similar S matrix to that derived above

In [ ]:
d = 200
zeta = 8
sketch = SparseSignSketch(d,zeta)
Xhat = sketch.sketch(X.T).T

First, just as a sanity check, we might ask whether we continue to preserve lengths - this time over rows.  

In [ ]:
plt.plot(np.linalg.norm(X,axis=1),np.linalg.norm(Xhat,axis=1),'k.')

Yes!  This is important because fundamentally, what k-means is doing is computing distances (in feature space) from any point to a cluster center, and so its important that those distances are preserved.  We will use scikit-learn's built in clustering algorithm, which has effective methods for initialization, etc. - no need to reinvent the wheel here.  We will compare the clustering algorithm's behavior on both the full and reduced dataset.  

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 10

models = []
for data in [X,Xhat]:
    model = KMeans(n_clusters)
    model.fit(data)
    models.append(model)

In general, it's difficult to analyze the performance of an unsupervised classification algorithm.  However, in this case we do have labels.  Assuming that our clustering is reasonably effective at dividing different digits into different groups, we can map from a true label $k$ (which the model has never seen) to the model's arbitrary class by evaluating which cluster has the most $k$'s in it.

In [ ]:
from scipy.stats import mode
desired_index = 3

def sample_cluster_by_desired_digit(X,model,target,index):
    clusters = model.predict(X)
    cluster_index = mode(clusters[target==index]).mode
    return clusters == cluster_index

for X_,model in zip([X,Xhat],models):
    mask = sample_cluster_by_desired_digit(X_,model,mnist.target.astype(int),desired_index)      
    fig,axs = plt.subplots(nrows=4,ncols=4)
    for ax in axs.ravel():
        i = np.random.randint(mask.sum())
        ax.imshow(X[mask][i].reshape(28,28))

Just for the purposes of quantitative comparison, we can try something similar with logistic regression.  We'll split the data up into a training and test set and compute the test accuracy.

In [ ]:
from sklearn.linear_model import LogisticRegression

rand_ind = np.random.permutation(range(70000))
n_train = 60000

training_indices = rand_ind[:n_train]
test_indices = rand_ind[n_train:]

target = mnist.target.astype(int)

models = [LogisticRegression(tol=1e-3) for _ in range(2)]

accuracy = []

for X_,model in zip([X,Xhat],models):
    model.fit(X_[training_indices],target[training_indices])
    accuracy.append(np.mean(model.predict(X_[test_indices]) == target[test_indices]))
    
print(accuracy)

Despite having reduced our data by 1/7th via exclusively random projection, we have retained most of our test accuracy!  

### Randomized PCA
As a final example of randomized methods, we will explore an efficient mechanism to compute an approximate Principal Component Analysis of a (perhaps large) dataset.  This methodology is slightly more in depth (and more accurate) than the previous two.  For the PCA, it also relies upon the fact that a data set that lives in $\mathbb{R}^{n \times m}$ might actually lie on something close to a manifold, which is to say an embedded geometric structure of dimension $r$ where $r<m$ (for example a line in a 2d plane, or a line or plane in a 3D volume, etc).  PCA reveals this structure ex post facto - but if we can assume it a priori, then we can accelerate computation.  

### Low rankedness
Before we get back to the PCA, let's think a little bit about what a matrix does, and some structures that it might have.  First, let's consider some vectors.  Let's begin by generating and plotting a couple of random vectors.

In [ ]:
def vec_plot(y,c=None,ls='solid'):
    #if c is None:
        #c = 'black'
    origin = np.column_stack([[0, 0]]*y.shape[1])
    plt.plot(0,0,'ok')
    if c is not None:
        plt.quiver(*origin,*y,c,angles='xy', scale_units='xy', scale=1, ls=ls, facecolor='none', linewidth=1)
    else:
        plt.quiver(*origin,*y,angles='xy', scale_units='xy', scale=1, ls=ls, facecolor='none', linewidth=1)
    plt.axis('equal')
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.xlim(-3,3)
    plt.ylim(-3,3)
    
x = np.random.rand(2,3) * 2 - 1
c = np.linspace(0,1,x.shape[1])
vec_plot(x,c)

Now let's multiply these same vectors, and see what happens to them.

In [ ]:
A_full = np.random.randn(2,2)
vec_plot(A_full @ x, c,ls='dashed')

Obviously they get transformed - but how exactly?  Let's look at the matrix that did the multiplying.  

In [ ]:
print(A_full)
print(x[:,0])
print(A_full[:,0]*x[0,0] + A_full[:,1]*x[1,0])

The matrix multiplication formula for $A\mathbf{x}$ can be written as $a_1 x_1 + a_2 x_2$, where $a_i$ is the $i$-th column of $A$.  As such matrix multiplication is saying take $x_1$ of the first column of $A$ plus $x_2$ of the second column of $A$ and so on.  As such, one way of looking at a matrix is as a *basis* for some space (e.g. $\mathbb{R}^2$ or something else) - and the space is defined as the span of the columns of $A$ (the set of vectors produced by taking all possible linear combinations of the basis).  The act of matrix multiplication takes the coordinates of a point defined in terms of the basis $A$ and turns it into a basis defined in terms of the standard basis, which has a special matrix associated with it
$$
I = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}.
$$
This makes sense - if a vector that is represented via the standard basis is transformed to be represented in the standard basis, then nothing happens.  If a vector representing as the coefficients of a different basis are multiplied by a matrix, then we recover the representation of that vector in the standard basis.  Here's another example: let's say that we have as our basis.

In [ ]:
A = np.array([[np.sqrt(2)/2,-np.sqrt(2)/2],[np.sqrt(2)/2,np.sqrt(2)/2.]])
vec_plot(A)

If we multiply this matrix by the vector $[1,1]^T$, what we are asking for is the first basis vector plus the second basis vector represented in the standard basis.  This ought to be aligned with the vertical axis, and because the basis vectors are orthogonal and unit length, the norm of the vector ought not to change.   

In [ ]:
x = A @ np.array([[1],[1]])

vec_plot(x)
print(x)

What is the space for which $A$ is the basis?  Well, it should be pretty obvious that we can represent any vector in $\mathbb{R}^2$ as a linear combination of either the standard basis (columns of the identity) or the columns of $A$.  In fact, for most matrices in $\mathbb{R}^2$, this is the case. 

In [ ]:
x_rand = np.random.randn(2,100)
vec_plot(x_rand)
plt.figure()
vec_plot(A_full @ np.random.randn(2,100))

However, there are examples for which this is *not* the case.  For example, consider the following matrix, which we can also try multiplying by a bunch of random vectors

In [ ]:
A_bad = np.random.randn(2,1) @ np.random.randn(1,2)

vec_plot(x_rand)
plt.figure()
vec_plot(A_bad @ x_rand)

All these vectors are being mapped to a line!  Despite having two basis vectors that we could add up in different combinations to produce output vectors, adding those vectors always produced output vectors that would lie on a line.  Stated another way, the space spanned by the columns was of a lower dimension than the number of columns (you should convince yourself that this only happens when one column is a constant multiple of another).  We call a matrix for which this is the case "low rank".

For lots of applications, this is viewed as a negative property.  For example, such matrices are not invertible.  However, it also offers a path towards compression.  

To do this, let's first try to understand the line to which all of our vectors are being mapped.  These vectors are still in $\mathbb{R}^2$, but it's clear that they are uniquely indexed by some constant multiple of some vector $\mathbf{q}$ that forms a basis for the 1D subspace.  From that perspective, the coordinate of any datapoint is just a scalar, and these can be mapped back to $\mathbb{R}^2$ by multiplying by $\mathbf{q}$.  How do we get $\mathbf{q}$?  Well, we already have a bunch of copies of it scaled by different factors!  The scaling is non-unique, so let's choose to take one of unit length.

In [ ]:
y_bad = A_bad @ x_rand
q = (y_bad[:,0]/np.linalg.norm(y_bad[:,0])).reshape(-1,1)
vec_plot(A_bad @ x_rand)
plt.plot(*(q*np.array([-3,3])),'r-')

Clearly, this spans the space.  Now, to represent the dataset of vectors using only a single number, we need to know where - in terms of the standard basis on the 1-D subspace - the vectors $\mathbf{x}$ map to.  In fact this is is precisely what $\mathbf{q}^T$ does: take something represented in a particular 2D basis, and represent it in the standard basis in 1D.  As such, it ought to be the cases that
$$
A\mathbf{x} = \mathbf{q} \mathbf{q}^T A \mathbf{x} \; \forall \mathbf{x}.
$$

In [ ]:
plt.plot(q @ (q.T @ A_bad @ x_rand),A_bad @ x_rand,'k.')

This is lossless compression.  Now, instead of representing a dataset element as $\mathbf{y} = A \mathbf{x}$, we could represent it as $c=\mathbf{q}^T A \mathbf{x}$, which is a scalar for each datapoint.  When we need to recover $\mathbf{x}$, we can multiply it by $\mathbf{q}$. 
Note that since this holds for all $x$, this result also implies that
$$
A = \mathbf{q} (\mathbf{q}^T A).
$$

While we have applied this approach to a very simple case of a 1-D manifold immersed in a 2-D space, the same ideas generally hold for a $k$-dimensional manifold immersed in a $m$-dimensional space.  We can't visualize high dimensional spaces, but we can see this same structure emerge numerically.  The trick is in figuring out how to compute the basis for the manifold $\mathbf{q}$.  However, this is where randomization comes in - if we take the product $A S^T$, with $S^T$ defined as one of the random matrices described previously, then with high probability
$$
Y = A S^T
$$
spans the same space as $A$.  

In [ ]:
k = 20
m = 50

A = np.random.randn(m,k)/np.sqrt(k) @ np.random.randn(k,m)/np.sqrt(k)

sketch = SparseSignSketch(k,8)
Y = sketch.sketch(A.T).T

Unfortunately, we can't use $Y$ as $q$ directly because the columns aren't orthogonal or unit length (and so $q^T$ doesn't map to the standard basis on the manifold).  However, we can produce an orthogonal and normal basis (so-called "orthonormal") by applying a simple procedure called Gram-Schmidt Orthogonalization.  We will not delve into the numerics of this process - that is better suited for a numerical linear algebra course - but there are two important facts.  First, this procedure is available to us by calling 

In [ ]:
q,_ = np.linalg.qr(Y)

and this function takes $\mathcal{O}(mk^2)$ time.  Did this work?

In [ ]:
A

In [ ]:
c = q.T @ A
print(c.shape)
print(q @ c)

One final note: this process still works to create a reasonable approximation even when the manifold is only approximately a manifold - like when you have a lot of variance in one direction and only a little bit of variance in the other direction, but both are non-zero.

In [ ]:
L_bad = np.random.randn(2,2)
V,_ = np.linalg.qr(L_bad)
A_bad = V * np.array([1,0.25]) @ V.T

vec_plot(x_rand)
plt.figure()
vec_plot(A_bad @ x_rand)

x_r = np.random.randn(2,1)

q,_ = np.linalg.qr(A_bad @ x_r)
#q,_,_ = np.linalg.svd(A_bad @ x_r)
plt.plot(*q[:,0][:,np.newaxis]*np.array([-3,3]))


Note that its awkward to illustrate this with a space that's low dimensional enough to be plotted because it only takes two samples for any $\mathbf{q}$ to span the whole space, so there's not much utility here.

### Principal Component Analysis
Let's recall how PCA works.  Let's consider that we have a data matrix, which we'll call $X$.  For this example, we could either use MNIST again, or we could use the classic example of "eigenfaces" - which is the application of PCA to datasets of facial images.  

In [ ]:
from sklearn.datasets import fetch_lfw_people
lfw_people = fetch_lfw_people(min_faces_per_person=1, resize=0.4)
plt.imshow(lfw_people.data[0].reshape(50,37),cmap=plt.cm.grey)

This is a pretty big dataset.  Before we apply PCA there, let's recall the intuition.  Let's say that we've got some dataset:

In [ ]:
np.random.seed(0)
L = np.random.randn(2,2)
V,_ = np.linalg.qr(L)
A_corr = V * np.array([1,0.1]) @ V.T
X = (A_corr @ np.random.randn(2,1000)).T + np.array([2,1])
plt.scatter(*X.T)
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.axis('equal')

First, for the sake of simplicity, let's subtract the mean so that we're centered on the origin.

In [ ]:
mu = X.mean(axis=0)
plt.scatter(*(X - mu).T)
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.axis('equal')

In order to understand the spread in the data due to different factors, we would like to compute variances (or standard deviations).  To that end, we would like to change our basis so that the two dimensions independent from one another, i.e. 
$$
Z = X V,
$$
where $V$ is a rotation matrix (i.e. perpendicular and unit-length columns), such that the axes line up with the directions of maximum variability in the data.  In this basis, we can compute a diagonal variance matrix as 
$$
n \Lambda = Z^T Z = V^T X^T X V. 
$$
Because $V$ is a rotation matrix, $V^T$ is its inverse and so we seek orthonormal $V$ and diagonal $\Lambda$ such that
$$
V \Lambda V^T = \frac{X^T X}{n}. 
$$
We recognize $\frac{X^T X}{n}$ as the covariance matrix (assuming the data is centered, otherwise it would be $\frac{(X - E[X])^T(X-E[X])}{n}$), and the left side as an eigendecomposition - the eigenvectors are the ones that the covariance matrix doesn't rotate, and the eigenvalues are the variances.  The resulting covariance matrix is of size $m\times m$ and is symmetric (and positive definite, if that means anything to you).  We can compute its eigendecomposition in $\mathcal{O}(m^3)$ time (but with quite a significant leading order coefficient typically around 40).  Computing the covariance matrix takes $\mathcal{O}(n m^2)$ time (not unlike the least-squares example).   

In [ ]:
Lamda, V = np.linalg.eigh(((X-mu).T @ (X-mu))/X.shape[0])
mu = X.mean(axis=0)
plt.scatter(*(X - mu).T)
vec_plot(V*3*np.sqrt(Lamda))

plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.axis('equal')

This quite helpfully determines the axes along which the data varies independently (the principal components), and the size of the eigenvalues tells us which of these axes has the most variability.

In principle, we could do *exactly* the same procedure for the facial dataset.  However, in this case it has  $m=1850$ features and $n=13500$ data points.  As such both forming the covariance matrix *and* computing the eigendecomposition is pretty expensive for a large dataset such as this one.  Instead, we will use randomized methods to compute the first - say - 100 principal components, operating under the assumption that the data is indeed low rank - or that a judicious combination of the columns of the data matrix contain nearly as much information as the full dataset.  

The first order of business is to compute the covariance matrix.  We will use the same sketching approach as we did in linear regression to do this efficiently.  Here, we'll use a variant of the Hadamard sketch called the Discrete Cosine Transform Sketch, just because I was bored and wanted to see if it would work.    

In [ ]:
from scipy.fft import dct
class DiscreteCosineSketch:
    def __init__(self,d):
        self.d = d

    def sketch(self,A,b=None,seed=None):
        if seed is not None:
            np.random.seed(seed)
        n,m = A.shape
        indices = np.random.choice(n,self.d,replace=False)
        D = np.random.randint(0,2,n)*2 - 1
        scale = 1./np.sqrt(2*d)
        SA = scale*dct(D.reshape(-1,1)*A,axis=0)[indices]
        if b is not None:
            Sb = scale*dct(D.reshape(-1,1)*b.reshape(-1,1),axis=0)[indices].ravel()
            return SA,Sb
        else:
            return SA

In [ ]:
X = lfw_people.data - lfw_people.data.mean(axis=0)
Xbar = X - X.mean(axis=0)
Xbar /= Xbar.std()

n,d = Xbar.shape
sk = DiscreteCosineSketch(d)
Xhat = sk.sketch(Xbar)

Let's just double check that this thing really does preserve the length of the columns

In [ ]:
plt.plot(np.linalg.norm(Xbar,axis=0),np.linalg.norm(Xhat,axis=0),'k.')
plt.axis('equal')

Now, we can cheaply form the covariance matrix.  (NOTE: There are different orders in which this could be done - here we sketch the columns of the covariance matrix, but one could also sketch the columns of the already-sketched data matrix that we just produced.)

In [ ]:
C = Xhat.T @ Xhat / n

Now, let's sketch the columns (or rows, since it's a symmetric matrix)

In [ ]:
sketch_cols = DiscreteCosineSketch(500)
Y = sketch_cols.sketch(C).T

We now have a sketch of the column space of the covariance matrix, and we assume that this column-space lies on a low-dimensional manifold of size less than 500.  Under that assumption, we can use the same machinery that we derived above and assume that 
$$
C \approx Q Q^T C,
$$
where $Q$ is an orthonormal basis for the column space of the covariance matrix $C$.  Indeed, we can go one further - because $C$ is symmetric, its column and row spaces are the same, so
$$
C \approx Q Q^T C Q Q^T.
$$
We can show that this is mostly true empirically.  First, compute $Q$ from the sketch.

In [ ]:
Q,_ = np.linalg.qr(Y)

For convenience, let's compute the small inner term $Q^T C Q$

In [ ]:
B = Q.T @ C @ Q
print(B.shape)

Now let's plot the covariance against its doubly-sketched approximation

In [ ]:
fig,axs = plt.subplots(ncols=3)
fig.set_size_inches(18,12)
axs[0].imshow(C,vmin=0,vmax=1)
axs[1].imshow(Q @ B @ Q.T,vmin=0,vmax=1)
axs[2].imshow(Q @ B @ Q.T - C,vmin=-0.05,vmax=0.05,cmap=plt.cm.seismic)

This is really quite accurate.  Now for a cheap eigendecomposition, which is really quite straightforward.  Instead of taking the eigendecomposition of $C$, we'll take the eigendecomposition of $B$:  
$$
C \approx Q B Q^T = Q U \Lambda U^T Q^T.
$$
$\Lambda$ is a diagonal matrix of size $l \times l$, while $U$ is an orthonormal matrix of size $m \times l$.  If we define
$$
V = Q U,
$$
then $V$ is another orthonormal matrix (a rotation applied to a rotation is still a rotation after all).  We thus have
$$
C \approx V \Lambda V^T,
$$
with $V$ orthonormal and $\Lambda$ diagonal, which is *by definition* and eigendecomposition of $C$, and thus the PCA!  

In [ ]:
Lamda,U = np.linalg.eigh(B)

# eigh produces eigenvalues in ascending rather than the standard descending order - flip em around
Lamda = Lamda[::-1]
U = U[:,::-1]

V = Q @ U

Let's look at the first 9 principal components.

In [ ]:
fig,axs = plt.subplots(nrows=3,ncols=3)
fig.set_size_inches(12,12)
axs = axs.ravel()
for i,ax in enumerate(axs):
    ax.imshow(V[:,i].reshape(50,37),cmap=plt.cm.gray)

How expensive was this to compute?  First we had to sketch the rows, which required $\mathcal{O}(n \log n m)$ operations (because of the sketch that we used.  Different embeddings may have been cheaper).  Next we had to compute the covariance matrix, which cost $\mathcal{O})(d^2 m)$, where $d$ was, in this case quite a bit smaller than $m$.  Next we had to sketch it, which cost $\mathcal{O}(m^2 \log m)$.  Next, we had to compute $Q$, which was $\mathcal{O}(l^2 m)$.  Finally, we computed the eigendecomposition of $B$, which cost $\mathcal{O}(l^3)$ operations.  Overall, the complexity was $\mathcal{O}((n\log n + m \log m + l^2) m)$.  Compare this to the standard approach, which would have been $\mathcal{O}((n + m) m^2)$.  This is quite a bit faster asymptotically, especially if we use sketch sizes that are smaller than $m$.  But is the whole thing accurate?  As it turns out, this problem is still not large enough to be prohibitive to compute a "normal" PCA.  Let's do that and compare the results.

In [ ]:
Lamda_true,V_true = np.linalg.eigh(Xbar.T @ Xbar/n)
Lamda_true = Lamda_true[::-1]
V_true = V_true[:,::-1]

In [ ]:
fig,axs = plt.subplots(nrows=3,ncols=3)
fig.set_size_inches(12,12)
axs = axs.ravel()
for i,ax in enumerate(axs):
    ax.imshow(np.column_stack((V[:,i].reshape(50,37)*np.sign(V[0,i]),V_true[:,i].reshape(50,37)*np.sign(V_true[0,i]))),cmap=plt.cm.gray)

The eigenvectors are very similar (although a bit noisy) - what about the variances?

In [ ]:
plt.semilogy(Lamda_true[:500])
plt.semilogy(Lamda[:500])

Note that we could now use this PCA for any number of downstream tasks - for example clustering or classification, for which the PCA-transformed data performs very nearly as well as the original (because PCA is close to lossless compression).  Assuming that a meaningful low dimensional structure exists in column space and the number of non-negligible principal components is actually small, these randomized methods allow for the application of PCA to *enormous* problems with very little accuracy loss.